# TLCUNET

## predict whole image

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import os
import torch
from tqdm import tqdm
import numpy as np
import tifffile as tif

from mymetrics import heightacc
from mymodel import TLCNetU

import matplotlib.pyplot as plt
import tifffile as tif
from os.path import join
import math
import time

In [2]:
!pip install spectral


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import matplotlib.pyplot as plt

In [3]:
# Setup device
device = 'cuda'

In [4]:
import rasterio as rio

In [5]:
from osgeo import gdal

In [6]:
# Setup Model
model = TLCNetU(n_classes=1).to(device)
if torch.cuda.device_count() > 1:
    model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))

In [7]:
#resume = cfg["training"]["resume"]
resume = r'runs\tlcnetu_zy3bh\finetune.tar'
if os.path.isfile(resume):
    print("=> loading checkpoint '{}'".format(resume))
    checkpoint = torch.load(resume)
    model.load_state_dict(checkpoint['state_dict'])
    # optimizer.load_state_dict(checkpoint['optimizer'])
    print("=> loaded checkpoint '{}' (epoch {})"
             .format(resume, checkpoint['epoch']))
else:
    print("=> no checkpoint found at resume")
    print("=> Will start from scratch.")

model.eval()

=> loading checkpoint 'runs\tlcnetu_zy3bh\finetune.tar'
=> loaded checkpoint 'runs\tlcnetu_zy3bh\finetune.tar' (epoch 298)


TLCNetU(
  (uencoder1): Uencoder(
    (conv1): unetConv2(
      (conv1): Sequential(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (conv2): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
    )
    (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): unetConv2(
      (conv1): Sequential(
        (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (conv2): Sequential(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(32, eps=1e

In [8]:
def predict_whole_image(model, image, r, c, grid=400):
    '''
    image: n,r,c,b  where n = 1
    model: FCN
    non-overlap
    '''
    #grid=400
    n, b, rows, cols = image.shape
    # rows=math.ceil(r/grid)*grid
    # cols=math.ceil(c/grid)*grid
    # image_= np.pad(image,((0,0), (0,0), (0,rows-r), (0,cols-c)),'symmetric')
    # weight = np.ones((rows, cols))
    res = np.zeros((rows, cols),dtype=np.float32)
    seg = np.zeros((rows, cols), dtype=np.uint8)
    num_patch= len(range(0,rows,grid))*len(range(0,cols,grid))
    print('num of patch is',num_patch)
    k=0
    for i in range(0,rows, grid):
        for j in range(0, cols, grid):          
            patch = image[0:,0:,i:i+grid,j:j+grid].astype('float32')
            if np.max(patch.flatten())<=10e-8:
                continue
            start=time.time()
            patch = torch.from_numpy(patch).float()
            
            # normalization 
            patch = patch/1000.0

            pred = model(patch.to(device))
            pred0 = pred[0].cpu().detach().numpy() # height
            pred1 = pred[1].cpu().detach().numpy() # seg
            res[i:i+grid,j:j+grid] = np.squeeze(pred0)
            seg[i:i+grid,j:j+grid] = np.argmax(np.squeeze(pred1), axis=0) # argmax
            end=time.time()
            k=k+1
            #print('patch [%d/%d] time elapse:%.3f'%(k,num_patch,(end-start)))
    res = res[0:r,0:c].astype(np.float32)
    seg = seg[0:r,0:c].astype(np.uint8)
    return res, seg

## batch processing 

In [9]:
# read all city names
def read_filepath(filepath, filename):
    filelist=list()
    for root, dirs, files in os.walk(filepath):
        for name in files:
            if name.endswith(filename):
                filelist.append(join(root, name))
    filelist.sort()
    return filelist

In [11]:
filelistnew = [r'Z:\yinxcao\hongkong\mux_quacss1.tif']

In [11]:
grid =400

In [12]:
def loadenvi(file, band=1):
    dataset = gdal.Open(file)
    band = dataset.GetRasterBand(band)
    data = band.ReadAsArray()
    dataset = None
    band = None
    return data

In [13]:
data = loadenvi(filelistnew[0], band=1)
print(data.shape)
print(data.dtype)
data = None

(16800, 20800)
uint16


In [14]:
for file in filelistnew[:1]:
    # filepath
    idirname=os.path.dirname(file)
    predpath = join(idirname, 'pred')
    respath=join(predpath, 'predtlcnetu_200.tif') # model tlcnetu, epoch 200
    segpath=join(predpath, 'predtlcnetu_200_seg.tif')
    if os.path.exists(respath):
        print('file: %s already exist, then skip'%respath)
        continue
    if not os.path.exists(predpath):
        print('mkdir %s'%predpath)
        os.mkdir(predpath)
        
    print('process: %s', file)
    nadpath=join(idirname,'nads')
    fwdpath=join(idirname,'fwds')
    bwdpath=join(idirname,'bwds')
    # 1.read image
    band1 = loadenvi(file, band=1)
    r,c =band1.shape[:2]
    img=np.zeros((r,c, 7), dtype='uint16')
    for i in range(4):
        img[:, :, i] = loadenvi(file, i+1)
    img[:, :, 4]=loadenvi(nadpath)
    img[:, :, 5]=loadenvi(fwdpath)
    img[:, :, 6]=loadenvi(bwdpath)
    img = img.transpose(2, 0, 1) # C H W
    
    # img=img/10000
    img = np.expand_dims(img, axis=0) # 1 C H W
    
    rows=math.ceil(r/grid)*grid
    cols=math.ceil(c/grid)*grid
    img = np.pad(img,((0,0), (0,0), (0,rows-r), (0,cols-c)),'symmetric')

    # 2. predict
    starttime=time.time()
    res=predict_whole_image(model, img, r, c,  grid=400)
    endtime=time.time()
    print('success: %s, time is %f'%(file, endtime-starttime))

    img = 0 # release 
    
    # 3. export
    reffile = file+'8.tif'
    rastermeta = rio.open(reffile).profile
    rastermeta.update(dtype=res[0].dtype, count=1, compress='lzw')
    with rio.open(respath, mode="w", **rastermeta) as dst:
        dst.write(res[0],1)
    rastermeta.update(dtype=res[1].dtype, count=1, compress='lzw')
    with rio.open(segpath, mode="w", **rastermeta) as dst:
        dst.write(res[1],1)
#     tif.imsave(respath, res[0]) # building height
#     tif.imsave(segpath, res[1]) # building segmentation

process: %s D:\data\zy3height\ningbo\mux_quacs
num of patch is 2184
success: D:\data\zy3height\ningbo\mux_quacs, time is 40.753537
